<a target="_blank" href="https://colab.research.google.com/github/jmanuelc87/nmp-autoavanza/blob/main/notebooks/MontePiedad_Rules.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [74]:
import os
import base64
import functools

from pydantic import BaseModel, Field

from operator import itemgetter

from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableLambda
from langchain_core.runnables.branch import RunnableBranch
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.image import UnstructuredImageLoader

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ['OPENAI_ORG_ID'] = '***'
    os.environ['OPENAI_PROJECT_ID'] = '***'
    os.environ['OPENAI_API_KEY'] = '***'

In [3]:
llm = ChatOpenAI(
    model="*",
    # model = "gpt-4.1-mini",
    temperature=0,
    base_url='http://localhost:1234/v1',
)

In [4]:
extract = [
    {
        "role": "system",
        "content": "Eres un asistente servicial, usando OCR extraes los attributos que se te indiquen de la imagen y verificas que sean correctos comparandolo con el documento {doc_name}.",
    },
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": "data:image/jpg;base64,{image}"}},
            {"type": "text", "text": "El documento {doc_name}: {page_content}"},
        ],
    },
]

In [5]:
def load_image(inputs):
    """Load image from file and encode it as base64."""
    image_path = inputs["filename"]
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    image_base64 = encode_image(image_path)
    return image_base64

In [6]:
def load_unstructured_image(args):
    loader = UnstructuredImageLoader(args["filename"])
    data = loader.load()
    return data.pop().page_content

In [7]:
loader1 = RunnableLambda(load_unstructured_image)
loader2 = RunnableLambda(load_image)

### Extraccion Factura

In [8]:
file_url = "https://drive.google.com/file/d/1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6"
image1_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 1_TK 62853-1 FAC_FRENTE_otsu.jpg"

In [9]:
prompt_extraction = ChatPromptTemplate(messages=extract)

In [10]:
class Factura(BaseModel):
    nombre_cliente: str = Field(description="El nombre del cliente")
    niv: str = Field(description="El numero de serie, NIV o VIN del vehiculo")
    marca: str = Field(description="La marca del vehiculo")
    modelo: str = Field(description="El modelo del vehiculo")
    version: str = Field(description="La version del vehiculo")
    year: int = Field(description="El año del vehiculo")
    no_motor: str = Field(description="El numero de serie del motor")

In [11]:
llm_with_fac_structured_output = llm.with_structured_output(Factura)

In [12]:
def my_print(x):
    print("data", x)
    return x

In [13]:
extract_fac_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_fac_structured_output
)

In [14]:
factura1 = extract_fac_chain.invoke(input={
    "filename": image1_path,
    "doc_name": "Factura",
})

In [15]:
print(factura1)

nombre_cliente='RODRIGUEZ ELIZONDO FRANCISCO' niv='24108' marca='TOYOTA' modelo='HILUX' version='2019' year=2019 no_motor='MROEX8DD2K0186450'


### Extraccion Tarjeta de Circulacion

In [16]:
file_url = "https://drive.google.com/file/d/1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk"
image2_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 1_TK 62853-4 TC_FRENTE_hsv_vchannel_clahe_adaptativo.jpg"

In [17]:
class TarjetaCirculacion(BaseModel):
    nombre_cliente: str = Field(description="El nombre del cliente")
    vigencia: str = Field(description="La vigencia o fecha de vencimiento del documento")
    niv: str = Field(description="El numero de serie, NIV o VIN del vehiculo")
    placa: str = Field(description="El numero de placa del vehiculo")
    marca: str = Field(description="La marca del vehiculo")
    modelo: str = Field(description="El modelo del vehiculo")
    version: str = Field(description="La version del vehiculo")
    year: int = Field(description="El año del vehiculo")
    no_motor: str = Field(description="El numero de serie del motor")

In [18]:
llm_with_tc_structured_output = llm.with_structured_output(TarjetaCirculacion)

In [19]:
extract_tc_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_tc_structured_output
)

In [20]:
tc1 = extract_tc_chain.invoke(input={
    "filename": image2_path,
    "doc_name": "Tarjeta de Circulacion",
})

In [21]:
print(tc1)

nombre_cliente='FRANCISCO RODRIGUEZ ELIZONDO' vigencia='2019' niv='2TRA6018B6' placa='JW27321' marca='TOYOTA' modelo='HILUX 4 PUERTAS' version='PICK UP' year=2019 no_motor='MRDEXBDD2K0186450'


### Extraccion INE

In [22]:
file_url = "https://drive.google.com/file/d/1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1"
image3_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/INE_FRENTE/Caso 1_TK 62853-2 INE_FRENTE_hsv_vchannel_clahe_adaptativo.jpg"

In [23]:
class CredencialVotar(BaseModel):
    nombre: str = Field(description="El nombre de la persona")
    domicilio: str = Field(description="El domicilio de la persona")
    vigencia: str = Field(description="La vigencia de la credencial")

In [24]:
llm_with_cv_structured_output = llm.with_structured_output(CredencialVotar)

In [25]:
extract_cv_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_cv_structured_output
)

In [26]:
cv1 = extract_cv_chain.invoke(input={
    "filename": image3_path,
    "doc_name": "Credencial para Votar",
})

In [27]:
print(cv1)

nombre='Rodriguez Elizondo Francisco' domicilio='PRIV HIDALGO PONIENTE 548 INT.' vigencia='2018-2023'


# Reglas para validar documentos

Se definen ciertas reglas para validar documentos como se especifica a continuacion.

Para la tc del Auto se debe cumplir con los requisitos de autenticidad y datos correctos.
 - Se debe verificar la coincidencia del nombre del solicitante en la INE, Tarjeta de circulacion y Factura.
 - Se debe verificar el VIN es el mismo en la tarjeta de circulación y factura
 - Se debe comparar los datos del vehiculo (marca, modelo, año y version) en la tarjeta de circulacion y factura

In [86]:
def compare_doc_names(factura: Factura, tc: TarjetaCirculacion, cv: CredencialVotar) -> str:
    """Compara el nombre del cliente en los documentos Factura, Tarjeta de Circulacion y Credencial para Votar

    Args:
        factura (Factura): La factura del vehiculo
        tc (TarjetaCirculacion): La Tarjeta de circulacion
        cv (CredencialVotar): La Credencial para votar

    Returns:
        str: El resultado de la validacion es aprovado o rechazado 
    """
    names1 = set(factura['nombre_cliente'].lower().split(' '))
    names2 = set(tc['nombre_cliente'].lower().split(' '))
    names3 = set(cv['nombre'].lower().split(' '))
    names = [names1, names2, names3]
    # check for reducing the complexity O(n^2) to O(n)
    for i in range(3):
        for n1 in names[i]:
            # instead of using direct comparison test with a heuristic
            if n1 not in names[(i+1) % 3] or n1 not in names[(i+2) % 3]:
                return "El nombre no aparece correctamente en los tres documentos"
    return "Validado correctamente el nombre del cliente!"

In [87]:
def compare_doc_vin(factura: Factura, tc:TarjetaCirculacion) -> str:
    """Compara el numero de serie del vehiculo, NIV O VIN en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """
    if factura['niv'] == tc['niv']:
        return "El NIV aparace correctamente."
    return "El NIV es distinto. Favor de verficar"

In [88]:
validate = [
    {
        "role": "system",
        "content": "Eres un asistente servicial.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text":"La Factura\n{factura}\n\n"},
            {"type": "text", "text":"La Tarjeta de Circulacion\n{tc}\n\n"},
            {"type": "text", "text":"La Credencial para Votar\n{cv}\n\n"},
            {"type": "text", "text":"Realiza {comparation} documentos.\n"},
        ],
    }
]

In [89]:
validate_prompt = ChatPromptTemplate.from_messages(validate)

In [90]:
llm_with_tools = llm.bind_tools([compare_doc_names, compare_doc_vin])

In [91]:
def my_print(x):
    return x

out = RunnableLambda(my_print)

In [92]:
validations = ["la comparacion del nombre del cliente", "la comparacion del numero de serie o VIN"]

In [93]:
def validate_documents(inputs, validations):
    chain = validate_prompt | llm_with_tools | PydanticToolsParser(tools=[compare_doc_names, compare_doc_vin])

    response = []

    for validation in validations:
        result = chain.invoke(input={
            "comparation": validation,
            **inputs,
        })

        response.extend(result)

    return response

In [94]:
validation = RunnableLambda(functools.partial(validate_documents, validations=validations))

In [95]:
chain = {
    "factura": {
        "filename": itemgetter("factura"),
        "doc_name": lambda x: "Factura",
    }
    | extract_fac_chain,
    "tc": {
        "filename": itemgetter("tarjeta_circulacion"),
        "doc_name": lambda x: "Tarjeta de Circulacion",
    }
    | extract_tc_chain,
    "cv": {
        "filename": itemgetter("credencial_votar"),
        "doc_name": lambda x: "Credencial para Votar"
    }
    | extract_cv_chain,
} | validation

In [96]:
response = chain.invoke({
    "factura": image1_path,
    "tarjeta_circulacion": image2_path,
    "credencial_votar": image3_path,
})

In [97]:
response

['Validado correctamente el nombre del cliente!',
 'El NIV es distinto. Favor de verficar']